In [40]:
from sqlanalyzer import column_parser
import pandas as pd
import sqlparse
import re

In [41]:
from sqlanalyzer import column_parser
import pandas as pd
import sqlparse
import re


def delevel_query(query_list):

    sub_query = []
    pos_delete = [len(query_list)-1]
    pos_join = []
    for i, line in enumerate(query_list):
        if line.startswith('ORDER') or line.startswith('GROUP'):
            pos_delete.append(i)
        if line.startswith('FROM'):
            pos_join.append(i)
        if line.startswith('LEFT JOIN') or line.startswith('INNER JOIN') or line.startswith('FULL OUTER JOIN'):
            pos_join.append(i+1)

    pos_join.append(pos_delete[0])    


    sub_query = {}
    pos_join_list = iter(pos_join)
    next(pos_join_list)

    for i in range(len(pos_join)-1):
        if i < len(pos_join)-2:
            sub_query['sub_query_{}'.format(i)] = ' '.join(query_list[pos_join[i] : next(pos_join_list)-1])
        else:
            sub_query['sub_query_{}'.format(i)] = ' '.join(query_list[pos_join[i] : pos_join[-1]])
            
    return sub_query



def parse_sub_query(sub_query_list):
    sub_query = "\n".join(sub_query_list)
    sub_query = sub_query.lstrip('\n').lstrip(' ')
    
    formatter = column_parser.Parser(sub_query)
    formatted = formatter.format_query(sub_query)
    sub_query_list = formatted.split('\n')

    return sub_query_list


def has_child(sub_query_list):
    query_list = parse_sub_query(sub_query_list)
    query_list = delevel_query(query_list)
    return query_list, query_list != {}


In [42]:
query = """SELECT * FROM sfdc.accounts sfdc_accounts
LEFT JOIN (SELECT MAX(dt) FROM (SELECT dt FROM sfdc.oppty sfdc_oppty) LEFT JOIN (SELECT dt FROM sfdc.cases)) AS sfdc_cases ON sfdc_cases.dt = sfdc_accounts.dt
WHERE dt > '2020-04-03' AND dt < '2020-05-04' ORDER BY 1 GROUP BY 3 LIMIT 20
"""


## delevel_query

In [43]:
formatter = column_parser.Parser(query)
formatted_query = formatter.format_query(query)
cte_queries = formatter.parse_cte(formatted_query)

In [44]:
print(formatted_query)

SELECT *
FROM sfdc.accounts sfdc_accounts
LEFT JOIN
  (SELECT MAX(dt)
   FROM
     (SELECT dt
      FROM sfdc.oppty sfdc_oppty)
   LEFT JOIN
     (SELECT dt
      FROM sfdc.cases)) AS sfdc_cases ON sfdc_cases.dt = sfdc_accounts.dt
WHERE dt > '2020-04-03'
  AND dt < '2020-05-04'
ORDER BY 1
GROUP BY 3
LIMIT 20


In [45]:
# query_list = formatted_query.split('\n')
# has_subquery = True
# counter = 0
# while has_subquery:
#     counter += 1
#     sub_query_list, has_subquery = has_child(query_list)
#     query_list = sub_query_list
#     print(has_subquery, counter, '\n'.join(query_list),  '\n')

In [46]:
query_list_0 = formatted_query.split('\n')

In [47]:
query_list_0

['SELECT *',
 'FROM sfdc.accounts sfdc_accounts',
 'LEFT JOIN',
 '  (SELECT MAX(dt)',
 '   FROM',
 '     (SELECT dt',
 '      FROM sfdc.oppty sfdc_oppty)',
 '   LEFT JOIN',
 '     (SELECT dt',
 '      FROM sfdc.cases)) AS sfdc_cases ON sfdc_cases.dt = sfdc_accounts.dt',
 "WHERE dt > '2020-04-03'",
 "  AND dt < '2020-05-04'",
 'ORDER BY 1',
 'GROUP BY 3',
 'LIMIT 20']

In [48]:
sub_query_1 = delevel_query(query_list_0)['sub_query_1']

In [49]:
sub_query_1

"  (SELECT MAX(dt)    FROM      (SELECT dt       FROM sfdc.oppty sfdc_oppty)    LEFT JOIN      (SELECT dt       FROM sfdc.cases)) AS sfdc_cases ON sfdc_cases.dt = sfdc_accounts.dt WHERE dt > '2020-04-03'   AND dt < '2020-05-04' ORDER BY 1 GROUP BY 3"

In [50]:
formatter = column_parser.Parser(sub_query_1)
formatted_query = formatter.format_query(query)
formatted_query = formatted_query.lstrip('\n \(')
query_list_2 = formatted_query.split('\n')

In [51]:
query_list_2

['SELECT MAX(dt)',
 '   FROM',
 '     (SELECT dt',
 '      FROM sfdc.oppty sfdc_oppty)',
 '   LEFT JOIN',
 '     (SELECT dt',
 '      FROM sfdc.cases)) AS sfdc_cases ON sfdc_cases.dt = sfdc_accounts.dt',
 "WHERE dt > '2020-04-03'",
 "  AND dt < '2020-05-04'",
 'ORDER BY 1',
 'GROUP BY 3']

In [52]:
query_list_3 = parse_sub_query(query_list_2)
query_list_3 = delevel_query(query_list_3)

In [53]:
query_list_3

{'sub_query_0': 'FROM   (SELECT dt    FROM sfdc.oppty sfdc_oppty)',
 'sub_query_1': "  (SELECT dt    FROM sfdc.cases)) AS sfdc_cases ON sfdc_cases.dt = sfdc_accounts.dt WHERE dt > '2020-04-03'   AND dt < '2020-05-04' ORDER BY 1"}